In [ ]:
using Catan
using CatanLearning
using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
import MLJModelInterface
const MMI = MLJModelInterface

In [ ]:
function naive_model_proba(X)
    unscaled_feature = X[:CountSettlement] + X[:CountCity] + 2*(X[:HasLargestArmy] + X[:HasLongestRoad])
    return min(unscaled_feature / 10, 1.0)
end

MMI.@mlj_model mutable struct NaivePublicVictoryPoints <: MLJModelInterface.Deterministic
end

MMI.fit(::NaivePublicVictoryPoints, verbosity, data...) = (nothing, nothing, nothing)
MMI.predict(model::NaivePublicVictoryPoints, fitresult, new_data...) = naive_model_proba(new_data)

model = NaivePublicVictoryPoints()
naive_machine = machine(model, nothing, nothing) |> MLJ.fit!

In [ ]:
config_file = "feature_generation.toml"
(configs, player_configs, logger) = Catan.parse_configs(config_file)
rf_machine = CatanLearning.try_load_public_model_from_csv(player_configs)


In [ ]:
test_data_csv = "./data/test_public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)
y, X = unpack(df, ==(:WonGame));
df[1,:]

In [ ]:
MLJ.predict(naive_machine, nothing, df)

In [ ]:
header = collect(names(df))
vals = collect(df[1,:])
CatanLearning._predict_model_feature_vec(rf_machine, vals, header)

In [ ]:
header = collect(names(X))
feature_vals = collect(X[1,:])
data = reshape(feature_vals, 1, length(feature_vals))
#header = names(machine.data[1])
new_df = DataFrame(data, vec(header), makeunique=true)
new_df[!,:HasLargestArmy] = convert(Vector{Int8}, new_df[!,:HasLargestArmy])
new_df[!,:HasLongestRoad] = convert(Vector{Int8}, new_df[!,:HasLongestRoad])

In [ ]:
MLJ.predict(rf_machine, new_df)

In [ ]:
new_df